In [1]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.9MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 112kB 17.8MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 460kB 17.0MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=b09312b9ffa738f6e4a4e0945b7b0cec4dfa353a40d49ba7fc69565a910a060d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=5d778c306eb9e8e82e4bbd7c9cd6a1fbdaad87e6a26d0be94da07a3bc913a843
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=93da

In [1]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: dff003aa03e7d25df35a840b6f0660ae9675efb4
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [2]:
#GPU INFO
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue May  5 16:15:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar')

In [5]:
# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
# from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import wandb



noise file noisy_label_kv.txt generated with noise: 0.1



In [0]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model



def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if config.use_wandb == True:
        wandb.save(filename)

In [0]:
def scheduler(epoch: int):
    global lr
    lr = config.lr
    if epoch > config.start_epoch:
        lr = lr / 10.0
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Training
def train(epoch):
    global init
    net.train()
    tch_net.train()
    train_loss = 0
    correct = 0
    total = 0
    scheduler(epoch)


    # ramp up meta-learning rate and EMA decay
    if epoch <= config.param_epoch:
        u = epoch/config.param_epoch
        meta_lr = config.meta_lr*math.exp(-5*(1-u)**2)
    else:
        meta_lr = config.meta_lr
        config.eps = 0.999

    for step, (inputs, targets) in enumerate(train_loader):
        init_time = time.time()
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)

        class_loss = criterion(outputs, targets)
        class_loss.backward(retain_graph=True)
        

        if step > config.start_iter or epoch > 1:
        #if step > 0 or epoch > 0:

            # if step > config.mid_iter or epoch > 1:
            #     # config.eps = 0.999
            #     alpha = config.alpha 
            # else:
            #     u = (step - config.start_iter)/(config.mid_iter - config.start_iter)
            #     alpha = config.alpha*math.exp(-5*(1-u)**2)
            alpha = config.alpha

            if init:
                init = False
                for param, param_tch in zip(net.parameters(), tch_net.parameters()):
                    param_tch.data.copy_(param.data)
            else:
                for param, param_tch in zip(net.parameters(), tch_net.parameters()):
                    param_tch.data.mul_(config.eps).add_((1-config.eps), param.data)

            _, feats = pretrain_net(inputs, get_feat=True)
            tch_outputs = tch_net(inputs, get_feat=False)
            p_tch = F.softmax(tch_outputs, dim=1)
            p_tch.detach_()

            for i in range(config.num_fast):
                targets_fast = targets.clone()
                randidx = torch.randperm(targets.size(0))
                for n in range(int(targets.size(0)*config.perturb_ratio)):
                    num_neighbor = 10
                    idx = randidx[n]
                    feat = feats[idx]
                    feat.view(1, feat.size(0))
                    feat.data = feat.data.expand(targets.size(0), feat.size(0))
                    dist = torch.sum((feat-feats)**2, dim=1)
                    _, neighbor = torch.topk(dist.data, num_neighbor+1, largest=False)
                    targets_fast[idx] = targets[neighbor[random.randint(1, num_neighbor)]]

                fast_loss = criterion(outputs, targets_fast)

                grads = torch.autograd.grad(fast_loss, net.parameters(),
                                            create_graph=False,
                                            retain_graph=True,
                                            only_inputs=True)

                fast_weights = OrderedDict(
                    (name, param - meta_lr*grad)
                    for ((name, param), grad) in zip(net.named_parameters(), grads))

                fast_out = net.forward(inputs,fast_weights)

                logp_fast = F.log_softmax(fast_out,dim=1)
                #afegir canvis per iterative aquí
                consistent_loss = consistent_criterion(logp_fast, p_tch)
                consistent_loss = consistent_loss*alpha/config.num_fast
                consistent_loss.backward()

        optimizer.step()

        # train_loss += class_loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # Grab training results
        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%, time: %.3f'
              %(epoch, config.num_epochs, step+1, (len(train_loader.dataset)//config.batch_size)+1, class_loss.data.item(), 100.*correct/total,time.time() - init_time))
        sys.stdout.flush()



def valid(epoch, network):
    global best_acc
    network.eval()
    # val_loss = 0
    correct = 0
    total = 0
    for step, (inputs, targets) in enumerate(valid_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = network(inputs)
            loss = criterion(outputs, targets)

        # valid_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # Grab validation results
        valid_acc = 100. * correct / total
      # valid_results = ("| Epoch: {}/{}, val_loss: {:.3f}, val_acc: {:.3f}, "
      #                 "lr: {:.6f}".format(epoch,
      #                                     config.num_epochs,
      #                                     loss.data.item(),
      #                                     valid_acc,
      #                                     lr))
        # Grab validation results
        valid_results = ("| Epoch: {}/{}, val_loss: {:.3f}, val_acc: {:.3f}, ""lr: {:.6f}".format(epoch,config.num_epochs,loss.data.item(),valid_acc,lr))
        record.write(valid_results + '\n')
        record.flush()
       
        

    # Save checkpoint when best model
    if valid_acc > best_acc:
        best_acc = valid_acc
        print('| Saving Best Model ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '.pth.tar' 
        save_checkpoint({
            'state_dict': network.state_dict(),
            'best_acc': best_acc,
        }, save_point)
     
    wandb.log({'epoch': epoch, 'accy_val' : best_acc })

    return valid_results


def test():
    test_net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(valid_loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = test_net(inputs)
            loss = criterion(outputs, targets)

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total
    test_results = "| test_loss: {:.3f}, test_acc: {:.3f}".format(
        loss.data.item(), test_acc)
    record.write(test_results)
    record.flush()

    print(test_results)
    

In [0]:
def save_weights(epoch):
        print('| Saving Weights student ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(epoch) + '.pth.tar'
        save_checkpoint({'state_dict': net.state_dict(), }, save_point)

        print('| Saving Weights teacher ...', end="\r")
        save_point = config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(epoch) + '.pth.tar'
        save_checkpoint({'state_dict': tch_net.state_dict(), }, save_point)

In [9]:
# Checkpoint dir.
record = open(config.drive_dir + '/checkpoint/' + config.checkpoint + '_test.txt', 'w')
record.write('noise_rate=%s\n' % config.noise_rate)
record.flush()

# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()
tch_net = get_model()
pretrain_net = get_model()
test_net = get_model()

print('| load pretrained net. from checkpoint...')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + '.pth.tar')
pretrain_net.load_state_dict(checkpoint['state_dict'])

if use_cuda:
    net.cuda()
    tch_net.cuda()
    pretrain_net.cuda()
    test_net.cuda()
    cudnn.benchmark = True
pretrain_net.eval()

for param in tch_net.parameters():
    param.requires_grad = False
for param in pretrain_net.parameters():
    param.requires_grad = False

# Instantiate a loss function.
criterion = torch.nn.CrossEntropyLoss()
consistent_criterion = torch.nn.KLDivLoss()

# Instantiate an optimizer to train the model
optimizer = torch.optim.SGD(
    net.parameters(), lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay)

print('\nTraining model')
print('| Training Epochs = ' + str(config.num_epochs))
print('| Initial Learning Rate = ' + str(config.lr))
print('| Optimizer = ' + str(config.optimizer_type))



Model setup
| Building network: resnet34


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:122: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:124: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:125: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


| load pretrained net. from checkpoint...

Training model
| Training Epochs = 120
| Initial Learning Rate = 0.2
| Optimizer = SGD


In [0]:
init = True
best_acc = 0
for epoch in range(1, 1 + config.num_epochs):
    train(epoch)
    # Student validation
    std_results = valid(epoch, net)
    record.write(std_results + '\n')
    print(std_results)
    # Teacher validation
    tch_results = valid(epoch, tch_net)
    record.write(tch_results + '\n')
    record.flush()
    print(tch_results)

    save_weights(epoch)


| Epoch: 1/120, val_loss: 2.159, val_acc: 22.400, lr: 0.200000
| Epoch: 1/120, val_loss: 23.725, val_acc: 0.000, lr: 0.200000


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


| Epoch [  2/120] Iter[  1/352]		Loss: 2.2991 Acc@1: 14.844%, time: 3.439

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


| Epoch: 2/120, val_loss: 1.494, val_acc: 25.660, lr: 0.200000
| Epoch: 2/120, val_loss: 1.761, val_acc: 33.620, lr: 0.200000
| Epoch [  3/120] Iter[352/352]		Loss: 1.6831 Acc@1: 31.298%, time: 1.228| Epoch: 3/120, val_loss: 2.261, val_acc: 29.520, lr: 0.200000
| Epoch: 3/120, val_loss: 1.703, val_acc: 41.020, lr: 0.200000
| Epoch [  4/120] Iter[352/352]		Loss: 1.5851 Acc@1: 37.660%, time: 1.217| Epoch: 4/120, val_loss: 1.955, val_acc: 39.060, lr: 0.200000
| Epoch: 4/120, val_loss: 1.271, val_acc: 47.980, lr: 0.200000
| Epoch [  5/120] Iter[352/352]		Loss: 1.8387 Acc@1: 42.727%, time: 1.223| Epoch: 5/120, val_loss: 2.051, val_acc: 44.320, lr: 0.200000
| Epoch: 5/120, val_loss: 0.925, val_acc: 54.040, lr: 0.200000
| Epoch [  6/120] Iter[352/352]		Loss: 1.5219 Acc@1: 47.873%, time: 1.216| Epoch: 6/120, val_loss: 1.451, val_acc: 49.700, lr: 0.200000
| Epoch: 6/120, val_loss: 0.585, val_acc: 60.460, lr: 0.200000
| Epoch [  7/120] Iter[352/352]		Loss: 1.1587 Acc@1: 52.938%, time: 1.211| Epo

wandb: ERROR Error uploading "MLNT_r10_ResNet34_BasicBlock.pth.tar": CommError, /tmp/tmpat5keti4wandb/MLNT_r10_ResNet34_BasicBlock.pth.tar is an empty file


| Epoch: 21/120, val_loss: 0.508, val_acc: 88.940, lr: 0.200000
| Epoch [ 22/120] Iter[352/352]		Loss: 1.0617 Acc@1: 77.780%, time: 1.214| Epoch: 22/120, val_loss: 0.804, val_acc: 66.260, lr: 0.200000
| Epoch: 22/120, val_loss: 0.474, val_acc: 89.340, lr: 0.200000
| Epoch [ 23/120] Iter[352/352]		Loss: 0.8012 Acc@1: 78.060%, time: 1.220| Epoch: 23/120, val_loss: 0.565, val_acc: 76.280, lr: 0.200000
| Epoch: 23/120, val_loss: 0.531, val_acc: 89.780, lr: 0.200000
| Epoch [ 24/120] Iter[352/352]		Loss: 0.6781 Acc@1: 78.267%, time: 1.227| Epoch: 24/120, val_loss: 0.638, val_acc: 80.080, lr: 0.200000
| Epoch: 24/120, val_loss: 0.528, val_acc: 89.940, lr: 0.200000
| Epoch [ 25/120] Iter[352/352]		Loss: 0.8825 Acc@1: 78.684%, time: 1.235| Epoch: 25/120, val_loss: 0.673, val_acc: 79.640, lr: 0.200000
| Epoch: 25/120, val_loss: 0.528, val_acc: 90.000, lr: 0.200000
| Epoch [ 26/120] Iter[352/352]		Loss: 0.9400 Acc@1: 78.818%, time: 1.232| Epoch: 26/120, val_loss: 0.603, val_acc: 75.260, lr: 0.20

In [15]:
print('\nTesting model')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])
test()



Testing model
| test_loss: 0.158, test_acc: 92.320


In [22]:
#This code is useful when the code has crashed without getting to the maximum of epochs (due to google collab session termination)
#essentially we reload weights and following continue the training process

EPOCH = 39 #Input the last epoch computed

print('Load Student')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(EPOCH) + '.pth.tar' )
net.load_state_dict(checkpoint['state_dict'])

print('Load Teacher')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(EPOCH) + '.pth.tar' )
tch_net.load_state_dict(checkpoint['state_dict'])

Load Student
Load Teacher


<All keys matched successfully>

In [0]:
init = False
best_acc = 0
for epoch in range(EPOCH, 1 + config.num_epochs):
    train(epoch)
    # Student validation
    std_results = valid(epoch, net)
    record.write(std_results + '\n')
    print(std_results)
    # Teacher validation
    tch_results = valid(epoch, tch_net)
    record.write(tch_results + '\n')
    record.flush()
    print(tch_results)

    save_weights(epoch)

print('\nTesting model')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])
test()

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2247: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


| Epoch: 39/120, val_loss: 0.318, val_acc: 77.560, lr: 0.200000
| Epoch: 39/120, val_loss: 0.127, val_acc: 92.340, lr: 0.200000
| Epoch [ 40/120] Iter[352/352]		Loss: 1.0688 Acc@1: 81.371%, time: 1.169

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f7296d752e8>>
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f7296d752e8>>
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f7296d752e8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
  File "/usr/local/lib/python3.6/dist-packages/torch/utils

| Epoch: 40/120, val_loss: 0.649, val_acc: 77.540, lr: 0.200000
| Epoch: 40/120, val_loss: 0.125, val_acc: 92.400, lr: 0.200000
| Epoch [ 41/120] Iter[352/352]		Loss: 0.7597 Acc@1: 81.709%, time: 1.165| Epoch: 41/120, val_loss: 0.725, val_acc: 77.360, lr: 0.200000
| Epoch: 41/120, val_loss: 0.126, val_acc: 92.120, lr: 0.200000
| Epoch [ 42/120] Iter[352/352]		Loss: 0.9182 Acc@1: 81.813%, time: 1.166| Epoch: 42/120, val_loss: 0.251, val_acc: 76.840, lr: 0.200000
| Epoch: 42/120, val_loss: 0.139, val_acc: 92.500, lr: 0.200000
| Epoch [ 43/120] Iter[352/352]		Loss: 0.6576 Acc@1: 82.091%, time: 1.172| Epoch: 43/120, val_loss: 0.732, val_acc: 68.600, lr: 0.200000
| Epoch: 43/120, val_loss: 0.126, val_acc: 92.700, lr: 0.200000
| Epoch [ 44/120] Iter[352/352]		Loss: 0.7685 Acc@1: 82.191%, time: 1.169| Epoch: 44/120, val_loss: 0.298, val_acc: 80.020, lr: 0.200000
| Epoch: 44/120, val_loss: 0.144, val_acc: 92.580, lr: 0.200000
| Epoch [ 45/120] Iter[352/352]		Loss: 0.4432 Acc@1: 82.427%, time: 

In [0]:
# DISCARDING SAMPLES

In [0]:
# RELOAD NETWORKS
EPOCH = 120 #Input the last epoch computed

print('Load Student')
checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_student_' + str(EPOCH) + '.pth.tar' )
net.load_state_dict(checkpoint['state_dict'])

print('Load Teacher')

checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + '_teacher_' + str(EPOCH) + '.pth.tar' )
tch_net.load_state_dict(checkpoint['state_dict'])

print('\nLoad Test')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
test_net.load_state_dict(checkpoint['state_dict'])

In [0]:
import pandas
def get_confidence_training(dataloader):
  correct = 0
  total = 0

  target_list = []
  pred        = []
  confidence  = []

  for batch_idx, (inputs, targets) in enumerate(dataloader):
      if use_cuda:
          inputs, targets = inputs.cuda(), targets.cuda()
      with torch.no_grad():
          outputs = test_net(inputs)
          loss = criterion(outputs, targets)  
          _, predicted = torch.max(outputs.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()
          
          conf = float(torch.max(outputs.softmax(dim = 1)))
          targ = int(targets.data)
          pd = int(predicted)

          confidence.append(conf)
          target_list.append(targ)
          pred.append(pd)

  # Grab dataframe
  df = pandas.DataFrame()
  df["target_list"] = target_list
  df["pred"] = pred
  df["confidence"] = confidence

  return(df)

In [0]:
#Those parameters are key for discarding
pre_batch = config.batch_size
config.batch_size = 1
config.shuffle = False

# Get the dataloader with no shuffle
loader = dataloader.KeyDataLoader()
train_noshuffle_loader, _, _ = loader.run()


df = get_confidence_training(train_noshuffle_loader)

#get list of directories
train_df = pandas.read_csv(config.data_dir+config.train_dir, header=None)
train_df.columns = ["dir"]
train_df

#Merge directories with confidence
DF = df.merge(train_df,how = 'left', left_index = True, right_index = True)

#filter only training samples that have a high enough confidence by the softmax
tau = 0.01
df_filtered = DF[DF.confidence > tau]
df_filtered

df_filtered.to_csv(config.data_dir +'clean_train_key_list_iter2' + config.id + '.txt', header = None, index=False)
config.train_dir = 'clean_train_key_list_iter2' + config.id + '.txt'

config.shuffle = True
config.batch_size = pre_batch

# Get the filtered dataset
loader = dataloader.KeyDataLoader()
train_loader, _, _ = loader.run()

#Initialize mentor
mentor_net = get_model()
if use_cuda:
    mentor_net.cuda()

#Get the weights for the mentor
print('\nLoad Mentor')
checkpoint = torch.load(config.drive_dir + '/checkpoint/%s.pth.tar' % config.id)
mentor_net.load_state_dict(checkpoint['state_dict'])